<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>

<font color='blue'>
<br />
МАРК, рад приветствовать ВАС! <br />
<br />    

<br />
Меня зовут Николай. <br />И на этом этапе Вашего движения к новой профессии от меня будут комментарии по написанному коду. <br />Чтобы меньше "мусорить эфир" и чтобы максимально наглядно отделяться от основного кода, есть предложение условиться в некоторых обозначениях:<br /> <br />
<u><b>ТАКОЙ ШРИФТ</b></u> - всегда начало комментария <br />
<font color='green'>такой шрифт</font> - комментарии о том, что всё ОК <br/>
<font color='orange'>такой шрифт</font> - комментарии о том, что всё ОК по результату,<br> однако есть на что обратить внимание в плане применения техник, или есть способы сделать более короткий или быстрый код <br/>
<font color='red'>такой шрифт</font> - комментарии о том, что есть критичный момент, влияющий на бизнес-результат проекта.<br/>
</font><br /><br />
<font color='green'>Моей целью является не "уличить" в не знании, а просто высказать сверху Ваших знаний - свой опыт, что бы Вы могли использовать его для Вашего дальнейшего успеха)
<br>
</font>
<br>

</font>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4  

### Вывод

В столбцах 'days_employed' и 'total_income' имеются пропуски. В столбцах 'children' и 'days_employed' имеются нетипичные значения. В стобцах 'days_employed' и 'total_income' - вещественный тип данных.  

### Шаг 2. Предобработка данных

### Выявление и устранение артефактов

Приведем все строки к нижнему регистру.

In [2]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['purpose'] = df['purpose'].str.lower()

Выясним, какие артефакты есть в таблице:

In [3]:
print('Пол:\n',df['gender'].value_counts())
print('Максимальный возраст:',df['dob_years'].max())
print('Минимальный возраст:',df['dob_years'].min())

Пол:
 f      14236
m       7288
xna        1
Name: gender, dtype: int64
Максимальный возраст: 75
Минимальный возраст: 0


Артефакты - пол "xna" и нулевой возраст.

Поскольку лица младше 18 лет не могут получить кредит, уберем их из списка данных. Также удалим строку, где 'gender' = 'xna' (индекс этой строки - 10701)

In [4]:
print('Количесвто несовершеннолетних:',len(df[df['dob_years'] < 18]))
l = len(df)
df = df.drop(index = 10701)
data1 = df[df['dob_years'] >= 18].reset_index(drop = True)
print('Количество удаленных строк:',l-len(data1))
print(data1['gender'].value_counts())
print('Минимальный возраст:',data1['dob_years'].min())

Количесвто несовершеннолетних: 101
Количество удаленных строк: 102
f    14164
m     7259
Name: gender, dtype: int64
Минимальный возраст: 19


Обработаем столбец 'days_employed':

In [5]:
count1 = len(data1[data1['days_employed'] < 0])
count2 = len(data1[data1['days_employed']/365 >= data1['dob_years']])
count3 = data1['days_employed'].isna().sum()
print('Количесвто строк с трудовым стажем < 0:', count1)
print('Количество строк с трудовым стажем, большим, чем годы жизни:', count2)
print("Количество пустых значений в стобце 'days_employed'", count3)
print('Сумма вышеназванных значений:', count1 + count2 + count3)
print('Их процент от всех значений в столбце: {:.1%}'.format((count1 + count2 + count3)/len(data1)))

Количесвто строк с трудовым стажем < 0: 15831
Количество строк с трудовым стажем, большим, чем годы жизни: 3428
Количество пустых значений в стобце 'days_employed' 2164
Сумма вышеназванных значений: 21423
Их процент от всех значений в столбце: 100.0%


Из предыдущих расчетов ясно, что столбец 'days_employed' полностью непригоден для исследования, так как он состоит только из некорректных значений. 

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА 2</u>
</font>
<font color='green'>
МАРК, вот это сильный аргумент!!!!<br> 
 отличный разбор)<br>
    
Можно было, конечно. предположить, что отрицательные значения - просто ошибка со знаком. И надо взять модуль...<br>
Но сейчас я на Вашей стороне)

### Вывод

Были найдены и обработаны артефакты - пол "xna" и нулевой возраст. Столбец 'days_employed' содержит ошибки технического характера, появившиеся при некорректой выгрузке, либо при техническом сбое, либо при некорректной работе алгоритмов.

### Замена типов данных

Заменим вещественный тип данных в таблице на целочисленный.

In [6]:
#для корректной замены типов данных сначала заменим пропуски в столбцах на нетипичное значение
#чтобы потом заполнить эти пропуски групповыми медианами, заранее исключив дубликаты
print('Количество пропусков в каждом столбце:')
print(data1['days_employed'].isna().sum())
print(data1['total_income'].isna().sum())
data1 = data1.fillna(-1)


#замена типов данных
data1['total_income'] = data1['total_income'].astype(int)
data1['days_employed'] = data1['days_employed'].astype(int)
data1.info()
print(data1.head())

print('Количество нетипичных значений в каждом столбце:')
print(len(data1[data1['days_employed'] == -1]))
print(len(data1[data1['total_income'] == -1]))

Количество пропусков в каждом столбце:
2164
2164
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21423 entries, 0 to 21422
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null int64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null int64
purpose             21423 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   среднее             1   
3         3          -4124         

### Вывод

Для замены вещественных типов данных на целочисленные использовался метод .astype(), так как по сравнению с методом .to_numeric() есть возможность перевести данные в необходимый тип. <br>
Заполнение пропусков не привели к искажению результатов. Так как количество пропусков до замены равно количеству нетипичных значений после замены. <br>
Стоит обратить внимание на равное количество пропусков в столбцах 'days_employed' и 'total_income'.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОК

### Удаление дубликатов

In [7]:
print('Полных дубликатов в таблице:', data1.duplicated().sum())
data1 = data1.drop_duplicates().reset_index(drop = True)
print('Полных дубликатов в таблице после удаления:', data1.duplicated().sum())

print('Количество строк с нетипичными значениями после удаления дублей:',len(data1[data1['days_employed'] == -1]))

Полных дубликатов в таблице: 71
Полных дубликатов в таблице после удаления: 0
Количество строк с нетипичными значениями после удаления дублей: 2093


### Вывод

В случае данной таблицы могут иметь место только полные дубликаты (те строки, значения в которых совпадают в каждом столбце).
Возможные причины появление дубликатов: заявка в банк была отправлена несколько раз, либо же технический сбой. <br>
Также стоит заметить, что все дубли содержались в строках где имелись пропуски в столбцах 'days_employed' и 'total_income', так как количество таких строк уменьшилось ровно на числоудаленных дубликатов.

### Обработка пропусков

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='orange'>
обработка пропусков, всё-таки уже потом <br>
    
Обычно такая последовательность:<br>
устранение артефактов ---> удаление дублей ----> заполнение пропусков
<br>
</font>

Пропуски присутсвуют в двух стобцах: 'days_employed' и 'total_income'. Проверим, есть ли в этом взаимосвязь.

In [8]:
print("Число строк с нетипичным значением в столбце 'days_employed':", len(data1[data1['days_employed'] == -1]))
print("Число строк с нетипичным значением в столбце 'total_income':",len(data1[data1['total_income'] == -1]))
print('Число строк с нетипичным значением в обоих столбцах одновременно:',
      len(data1[(data1['days_employed'] == -1) & (data1['total_income'] == -1)]))

Число строк с нетипичным значением в столбце 'days_employed': 2093
Число строк с нетипичным значением в столбце 'total_income': 2093
Число строк с нетипичным значением в обоих столбцах одновременно: 2093



Отсюда можно сделать вывод, что это неслучайные пропуски, и они связаны.

Заполним пропуски групповой медианой, посчитанной на основе типа занятости. Посчитаем медианы для каждого типа занятости.

In [9]:
median1 = data1[(data1['total_income'] != -1) & (data1['income_type'] == 'сотрудник')]['total_income'].median()
print('Медиана дохода для типа занятости "сотрудник" -',median1)
median2 = data1[(data1['total_income'] != -1) & (data1['income_type'] == 'компаньон')]['total_income'].median()
print('Медиана дохода для типа занятости "компаньон" -',median2)
median3 = data1[(data1['total_income'] != -1) & (data1['income_type'] == 'пенсионер')]['total_income'].median()
print('Медиана дохода для типа занятости "пенсионер" -',median3)
median4 = data1[(data1['total_income'] != -1) & (data1['income_type'] == 'госслужащий')]['total_income'].median()
print('Медиана дохода для типа занятости "госслужащий" -',median4)
median5 = data1[(data1['total_income'] != -1) & (data1['income_type'] == 'предприниматель')]['total_income'].median()
print('Медиана дохода для типа занятости "предприниматель" -',median5)
print('\n')
print('Процент строк с пропуском от общего количества строк: {:.1%}'.format(len(data1[data1['total_income'] == -1]) / len(data1)))

Медиана дохода для типа занятости "сотрудник" - 142594.0
Медиана дохода для типа занятости "компаньон" - 172268.5
Медиана дохода для типа занятости "пенсионер" - 118556.5
Медиана дохода для типа занятости "госслужащий" - 150521.5
Медиана дохода для типа занятости "предприниматель" - 499163.0


Процент строк с пропуском от общего количества строк: 9.8%


Основываясь на данных, полученных выше, можно сделать вывод, что удаление строк с некорректными значениями внесет меньше ошибок в результат, чем заменять значения медианами.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='orange'>    
с 0 в "днях работы" и в "ежемесячном доходе" - на большое обсуждение<br>
1) чтобы на что-либо заменять нужны основания, аргументация (кстати Вы верно подметили, что пропуски симметричные - это знак для хранителей баз))))<br>
2) Тут нули будут слишком большим искажением реальной картины.<br>
Вариант: или удалять, или медиана<br>
3) Если медиана, то надо подумать какая: по всей выборке, или групповая. Ну, т.е. к какой группе относится пропуск, по этой же группе брать и медиану.<br>
например: пропуск ежемеясного дохода клиента заменять медианой на основе группы такой - его возрастной группы, типа занятости и образование

    
Что такое заполнение пропусков - это собственноручное внесение ошибок в результаты будущих анализов).<br>
И в этом деле надо выбирать, в каком случае ошибка в анализе будет меньшей: если что-нибудь вставить, или если оставить как есть?<br>

Замена на 0 - это большое искажение нашей базы(((
<br>
</font>
<br>
</font>

In [10]:
print(data1['children'].value_counts())

 0     14021
 1      4792
 2      2039
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64


Количество строк, где параметр 'children' = 20 равно 75. Такое количество семей с количеством детей равном 20 свидетельствует о некорректном сборе данных. Заполним эти пропуски нетипичным значением "-1" и обработаем их в дальнейшем.

In [11]:
def change(children):
    if children == 20:
        return -1
    return children

data1['children'] = data1['children'].apply(change)
print(data1['children'].value_counts())

 0    14021
 1     4792
 2     2039
 3      328
-1      122
 4       41
 5        9
Name: children, dtype: int64


Удалим строки со значением -1 в столбцах 'days_employed' и 'total_income'.

In [12]:
data = data1[(data1['days_employed'] != -1) & (data1['total_income'] != -1)]
print(len(data))

19259


### Вывод

Пропуски удалены. Были встречены неслучайные пропуски и ошибки в сборе данных. 


<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='orange'>
Марк, да я и сам такой - стараюсь не делать лишней работы))))<br>
    
Но этот проект - обучающий, поэтому, всё-таки, важно просмотреть все возможные "косяки" данных<br>
В этом-то и весь "цимус", как говорится. <br>
1) отрицательный стаж, стаж в 1100 лет<br>
2) пол, образование<br>
3) нулевой возраст...<br>
    
В общем много тут есть. И в таких проектах это надо увидеть))))<br>
<br>
</font>

### Обработка дубликатов

Проверим, не появились ли новые дубликаты после заполнения пропусков:

Найдем полные дубликаты:

In [13]:
print('Полных дубликатов в таблице:', data.duplicated().sum())

Полных дубликатов в таблице: 0


<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
здесь -ОК, согласен)

### Вывод

Дубликаты не появились.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='orange'>
С дубликатами - ок, здорово. Отмечу только 
Вопрос по последовательность действий:<br>
Что было сделано:<br>
сначала проведена работа с пропусками (допустим если бы вставили медиану), а теперь - дубликаты..<br>
Скорее всего, логичнее действовать так:<br>
1) провести усиленную борьбу с артефактами( в данной работе это пол, дети, нулевой возраст, отрицательный стаж..)<br>
2) удалить дубликаты<br>
3) провести работу с пропусками<br>
    
В такой последовательности замена пропуском будет без дополнительных смещений за счёт дублей
<br>
</font>

### Лемматизация

Лемматизиурем строки в столбце с целью кредита 'purpose':

In [14]:
from pymystem3 import Mystem
m = Mystem()
data['purpose_lemmatize'] = data['purpose'].apply(m.lemmatize)
print(data['purpose_lemmatize'].value_counts())

[автомобиль, \n]                                          869
[свадьба, \n]                                             717
[сыграть,  , свадьба, \n]                                 688
[на,  , проведение,  , свадьба, \n]                       682
[операция,  , с,  , недвижимость, \n]                     612
[покупка,  , коммерческий,  , недвижимость, \n]           594
[покупка,  , жилье, \n]                                   589
[покупка,  , жилье,  , для,  , сдача, \n]                 586
[жилье, \n]                                               583
[операция,  , с,  , коммерческий,  , недвижимость, \n]    580
[строительство,  , жилой,  , недвижимость, \n]            577
[операция,  , с,  , жилье, \n]                            573
[покупка,  , свой,  , жилье, \n]                          573
[недвижимость, \n]                                        570
[покупка,  , жилье,  , для,  , семья, \n]                 569
[строительство,  , недвижимость, \n]                      561
[операци

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Вывод

Столбец 'purpose' лемматизирован. Результат сохранен в новый столбец 'purpose_lemmatize'.

### Категоризация данных

Категоризируем заявки по целям кредита:

In [15]:
def purpose_group(row):
    if 'свадьба' in row:
        return 'свадьба'
    if 'автомобиль' in row:
        return 'сделки с автомобилем'
    if 'образование' in row:
        return 'образование'
    if ('жилье' in row) or ('недвижимость' in row):
        return 'операции с недвижимостью'
    return ''

data['purpose_group'] = data['purpose_lemmatize'].apply(purpose_group)
print(data['purpose_group'].value_counts())

операции с недвижимостью    9718
сделки с автомобилем        3875
образование                 3579
свадьба                     2087
Name: purpose_group, dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
согласен с подходом)

Создадим классификацию по возрасту: <br>
<br>
По новой возрастной классификации ВОЗ:<br>
от 18 до 44 лет — молодой возраст,<br> 
44—60 лет — средний возраст,<br> 
60—75 лет — пожилой возраст,<br> 
75—90 лет — старческий возраст,<br>
после 90 — долгожители.<br>
<br>
Создадим функцию для классификации.<br>
Поскольку строки с возрастом меньше 18 удалены, в функцию этот возраст передаваться не будет.<br>

In [16]:
def age_group(age):
    if (age >= 18) & (age < 44):
        return 'молодой возраст'
    if (age >= 44) & (age < 60):
        return 'средний возраст'
    if (age >= 60) & (age < 75):
        return 'пожилой возраст'
    if (age >= 75) & (age < 90):
        return 'старческий возраст'
    return 'долгожитель'

Выполним проверку функции:

In [17]:
print('25:', age_group(25))
print('50:', age_group(50))
print('64:', age_group(64))
print('75:', age_group(75))
print('92:', age_group(92))

25: молодой возраст
50: средний возраст
64: пожилой возраст
75: старческий возраст
92: долгожитель


In [18]:
data['age_group'] = data['dob_years'].apply(age_group)
print(data['age_group'].value_counts())

молодой возраст       10078
средний возраст        6926
пожилой возраст        2254
старческий возраст        1
Name: age_group, dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Создадим классификацию по количеству детей:
0 детей - бездетные,
1-2 ребенка - малодетные,
3-4 ребенка - среднедетные,
5 и более детей - многодетные.

Создадим функцию для классификации.

In [19]:
def family_group(children):
    if children == 0:
        return 'бездетные'
    if (children >= 1) & (children < 3):
        return 'малодетные'
    if (children >= 3) & (children < 5):
        return 'среднедетные'
    if children >= 5:
        return 'многодетные'
    return 'нет данных'

#создание столбца 'family_group' с классификацией
data['family_group'] = data[data['children'] != 20]['children'].apply(family_group)
#узнаем, как функция выполнила работу
print(data['children'].value_counts())
print(data['family_group'].value_counts())
print('Количество пропусков в новом стобце:',data['family_group'].isna().sum())

 0    12648
 1     4328
 2     1838
 3      293
-1      110
 4       34
 5        8
Name: children, dtype: int64
бездетные       12648
малодетные       6166
среднедетные      327
нет данных        110
многодетные         8
Name: family_group, dtype: int64
Количество пропусков в новом стобце: 0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Создадим более сложную классификацию по доходу:
Малоимущие - 10000 на человека
Беднейшие - 15000 на человека
Бедные - 18000 на человека
Выше бедности - 36000 на человека
Нижний средний класс - 100000 на человека
Предсредний класс - 150000  на человека
Средний класс - 250000  на человека
Верхний средний класс - 500000  на человека
Состоятельные - 1000000 на человека
Богатые - до бесконечности.

(Согласно данным за июнь 2018 года https://zen.yandex.ru/media/dogecrypto/kak-poschitat-k-kakomu-klassu-vy-otnosites-po-urovniu-dohoda-5b150b5aea0fe700a848e1b1)

Создадим классификацию по доходу с учетом количества человек в семье и семейным положением:

Чтобы посчитать количество дохода на человека в семье, посчитаем сумму человек в семье и разделим весь доход на это число.

In [20]:
print(data.groupby('family_status_id')['family_status'].unique())

def income_per_man(row):
    children = row['children']
    family_status_id = row['family_status_id']
    total_income = row['total_income']
    family_count = 1   #в семье как минимум один человек (чтю строку данных мы исследуем)
    if children != -1:    #если в столбце ['children'] корректные данные 
        if (family_status_id == 0) or (family_status_id == 1):   #если человек женат/замужем или в гражданском браке
            family_count = family_count + 1 + children
            return int(total_income / family_count)
        else:
            family_count = family_count + children
            return int(total_income / family_count)
    else:   #если в столбце ['children'] некорректные данные 
        if (family_status_id == 0) or (family_status_id == 1):  
            family_count = family_count + 1
            return int(total_income / family_count)
        else:   
            return int(total_income)

family_status_id
0          [женат / замужем]
1         [гражданский брак]
2           [вдовец / вдова]
3                [в разводе]
4    [не женат / не замужем]
Name: family_status, dtype: object


Применим функцию ко всему датафрейму и создадим новый столбец, в котором будет храниться доход на человека:

In [21]:
data['income_per_man'] = data.apply(income_per_man, axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Теперь создадим функцию, которая будет классифицировать доход по новому стобцу:

In [22]:
def income_group(income_per_man):
    if income_per_man < 10000:
        return 'ниже прожиточного минимума'
    if (income_per_man >= 10000) & (income_per_man <= 15000):
        return 'малоимущие'
    if (income_per_man > 15000) & (income_per_man <= 18000):
        return 'беднейшие'
    if (income_per_man > 18000) & (income_per_man <= 36000):
        return 'бедные'
    if (income_per_man > 36000) & (income_per_man <= 100000):
        return 'выше бедности'
    if (income_per_man > 100000) & (income_per_man <= 150000):
        return 'нижний средний класс'
    if (income_per_man > 150000) & (income_per_man <= 250000):
        return 'предсредний класс'
    if (income_per_man > 250000) & (income_per_man <= 500000):
        return 'средний класс'
    if (income_per_man > 500000) & (income_per_man <= 1000000):
        return 'верхний средний класс'
    if (income_per_man > 1000000) & (income_per_man <= 2500000):
        return 'состоятельные'
    return 'богатые'


<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
таааак, интересный подход)

In [23]:
#создание столбца 'income_types' с классификацией
data['income_group'] = data['income_per_man'].apply(income_group)
#узнаем, как функция выполнила работу
print(data['income_group'].value_counts())
print('Количество пропусков в новом стобце:',data['income_group'].isna().sum())
print(data[data['income_group'] == 'состоятельные'])

выше бедности                 10751
нижний средний класс           3198
бедные                         2588
предсредний класс              1806
средний класс                   562
беднейшие                       169
малоимущие                      117
верхний средний класс            46
ниже прожиточного минимума       19
состоятельные                     3
Name: income_group, dtype: int64
Количество пропусков в новом стобце: 0
       children  days_employed  dob_years education  education_id  \
7404          0          -8626         39   среднее             1   
12326         0          -1477         44    высшее             0   
20642         0          -4719         61   среднее             1   

               family_status  family_status_id gender income_type  debt  \
7404               в разводе                 3      f   сотрудник     0   
12326        женат / замужем                 0      m   компаньон     0   
20642  не женат / не замужем                 4      f   сотрудник 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Вывод

Данные категоризированы по четырем параметрам: по уровню дохода, по количеству детей, по целям кредита и по возрасту.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
без вопросов)

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [24]:
dp1 = data[data['family_group'] != 'нет данных']
dp1 = dp1.pivot_table(index = ['family_group'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
dp1 = dp1.fillna(0)
dp1['ratio'] = 1 - dp1[1]/dp1[0]
print(dp1.sort_values(by = 'ratio', ascending = False))
print(dp1['ratio'].median())

debt                0      1     ratio
family_group                          
многодетные       8.0    0.0  1.000000
бездетные     11701.0  947.0  0.919067
среднедетные    302.0   25.0  0.917219
малодетные     5584.0  582.0  0.895774
0.9181426447391432


### Вывод

Зависимость между наличием детей и возвратом кредита в срок отсутствует. В среднем семьи вовзращают кредит в 91% случаев. 

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОК,

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
dp2 = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'dob_years', aggfunc = 'count')

dp2['ratio'] = 1 - dp2[1]/dp2[0]
print(dp2.sort_values(by = 'ratio', ascending = False))
print(dp2['ratio'].median())

debt                       0    1     ratio
family_status                              
вдовец / вдова           806   55  0.931762
в разводе                998   76  0.923848
женат / замужем        10256  842  0.917902
гражданский брак        3379  337  0.900266
не женат / не замужем   2257  253  0.887904
0.9179017160686428


### Вывод

Зависимость есть. Люди, потерявшие супруга/супругу по причине смерти либо же развода отдают кредит чаще, чем те, кто вообще не вступал в брак и не обременен обязательствами.


<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
кто не брал на себя ещё ответственности))

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
dp3 = data.pivot_table(index = ['income_group'], columns = 'debt', values = 'dob_years', aggfunc = 'count')
dp3 = dp3.fillna(0)
dp3['ratio'] = 1 - dp3[1]/dp3[0]
print(dp3.sort_values(by = 'ratio', ascending = False))
print('Среднее значение:',dp3['ratio'].mean())

debt                             0      1     ratio
income_group                                       
состоятельные                  3.0    0.0  1.000000
ниже прожиточного минимума    18.0    1.0  0.944444
беднейшие                    158.0   11.0  0.930380
нижний средний класс        2966.0  232.0  0.921780
средний класс                520.0   42.0  0.919231
предсредний класс           1658.0  148.0  0.910736
выше бедности               9868.0  883.0  0.910519
верхний средний класс         42.0    4.0  0.904762
бедные                      2360.0  228.0  0.903390
малоимущие                   103.0   14.0  0.864078
Среднее значение: 0.9209319216105204


### Вывод

Прямой, либо же обратной зависимости от уровня дохода не выявлено. Самый низкий класс по уровню дохода из нашей классификации - "ниже прожиточного минимума" - уверенно держится в середине рейтинга (91% возвратов вовремя). При этом следующий по уровню дохода класс - малоимущие - замыкает рейтинг (86,4%), а следующий после него класс "беднейшие" возглавляет рейтинг (не учитываем класс "состоятельные", так как слишком мало данных по этому классу) с процентом возвратов равным 93%.  

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
dp4 = data.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'dob_years', aggfunc = 'count')

dp4['ratio'] = 1 - dp4[1]/dp4[0]
print(dp4.sort_values(by = 'ratio', ascending = False))

debt                         0    1     ratio
purpose_group                                
операции с недвижимостью  9006  712  0.920942
свадьба                   1931  156  0.919213
образование               3248  331  0.898091
сделки с автомобилем      3511  364  0.896326


### Вывод

Операции с недвижимостью - самая надежная цель кредита. По такому кредиту не имеют задолженности в 92% случаев.
Образование и сделки с автомобилем - возвращают практически с равной долей вероятности. Отличие от лидера всего лишь в 2%. 

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОК, сосгласен)

### Шаг 4. Общий вывод

Семейное положение и цели кредита - факторы, влияющие на вероятность его возврата. Закономерностей, связанных с уровнем дохода и количеством детей в семье не выявлено. Требуется более глубокое исследование. <br>

Следует обратить внимание на корректность выгрузки данных, поскольку важные данные рабочего стажа на 100% непригодны для исследования.

In [32]:
print('Процент возврата кредита по всей таблице: {:.1%}'.format(data[data['debt'] == 0]['debt'].count() / len(data)))

Процент возврата кредита по всей таблице: 91.9%


Наиболее ненадежные категории (категории, процент возврата которых меньше, чем по всей таблице):


In [36]:
columns = ['catrgory_name', 'ratio']
dt = [
    ['образование','0.898091'],
    ['сделка с автомобилем','0.898091'],
    ['бедные','0.903390'],
    ['малоимущие','0.864078'],
    ['не женат / не замужем','0.887904'],
    ['гражданский брак','0.900266'],['малодетные','0.895774']
]


rating = pd.DataFrame(data = dt, columns = columns)
print(rating.sort_values(by = 'ratio'))

           catrgory_name     ratio
3             малоимущие  0.864078
4  не женат / не замужем  0.887904
6             малодетные  0.895774
0            образование  0.898091
1   сделка с автомобилем  0.898091
5       гражданский брак  0.900266
2                 бедные  0.903390


In [ ]:
Слабая финансовая составляющая, очевидно, приводит к самому низкому проценту возврата кредитов.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
Коммерческие выводы ОК - мало, но логично. Только лучше именно в выводах давать более конкретные значения. Тогда будет интересно воспринимать  читаемое заказчиком. А так - кажется что работа совсем не значительная))))<br>
    
Также, стоит добавлять в выводы информацию об обнаруженных "косяках" или предположениях о косяках при выгрузке<br>
дубли, почему они могут быть; пропущенные значения и т.д.
<br>
</font>

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
МАРК, интереную Вы работу провели (если не считать блок предобработки данных).<br><br>
На мой взгляд, навыки которые Вы закрепляете с этой работой, наиважнейшие для дальнейших модельных дел<br>
Предобработка данных, чистка, проверка предположений - архиважно<br>
И у Вас много уже того, как должно быть.<br>
Особенно с аналитикой предпосылок и построении выводов<br>
Но также есть и то, что нужно ещё скорректировать<br>
<font color='red'>
1) оценить ещё раз этап, когда лучше проводить проверку дублирования<br>
Это ВАЖНЫЙ момент. Сейчас он не исказил инфо, потому что Вы нули подставили в пропуски, а так может повлиять.
2) Совсем не было времени у Вас на полнoценную предобработку данных((((<br>
Тут надо ещё поработать)<br>
3)Конечный вывод: Вы провели гораздо больший объём работ) отразите его в выводах, это важно для любого заказчика)
    
<font color='green'>
Конечно же много чего можно улучшать и корректировать по коду по самим вычислениям...<br>
Но не думаю, что это есть предмет обсуждения более важный,<br>
чем умение ставить аналитическую задачу и воплощать её в конечном решении<br>
А это этот навык Вы смогли показать в этой работе<br>
Остальное, уверен, будете набирать  каждым блоком обученя<br>

Скорректируйте, пожалуйста, то ,что я перечислил<br>

 <br><font color='green'>
БУДУ ЖДАТЬ ВАШИХ КОРРЕКТИРОВОК
  
<br>
</font>

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА 2</u>
</font>
<font color='green'>
МАРК, спасибо, что именно так отреагировали на мои комментарии<br>
То, что сейчас прочитал, было сильно:<br>
каждое действие - аргументировано. И двежение от блока к блоку выглядят более, чем логично<br>
Видно, что в этот раз потратили прилично времени. Но и результат от этого - отличная работа<br>
Спасибо)<br>

УСПЕШНОГО ДАЛЬНЕЙШЕГО ОБУЧЕНИЯ!

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.